<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#弄一個-Class-把跟-Pitcher-Arsenal-有關的放在這邊" data-toc-modified-id="弄一個-Class-把跟-Pitcher-Arsenal-有關的放在這邊-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>弄一個 Class 把跟 Pitcher Arsenal 有關的放在這邊</a></span></li></ul></div>

In [1]:
import pandas as pd
import numpy as np

In [2]:
csv_file_path = 'MLB_Player_Stats_CSV\\'

In [3]:
league_abb_dict = {'NL':['WSH', 'ATL', 'MIA', 'NYM', 'PHI', 'CHC', 'STL', 'PIT',
                         'CIN', 'MIL', 'COL', 'LAD', 'AZ', 'SF', 'SD'],
                   'AL':['NYY', 'TB', 'TOR', 'BOS', 'BAL', 'DET', 'CWS', 'CLE',
                         'MIN', 'KC', 'LAA', 'SEA', 'OAK', 'TEX', 'HOU']}

### 弄一個 Class 把跟 Pitcher Arsenal 有關的放在這邊

In [4]:
#----Process Pitcher Arsenal Data
class Pitcher_Arsenal:
    def __init__(self, year, player_type):
        self.year = year
        self.player_type = player_type
        csv_file_name = f'{csv_file_path}{self.year}{self.player_type}_pitcher_arsenal.csv'
        self.df = pd.read_csv(csv_file_name)
    
    def select_given_player(self, player_id):
        self.df_this_player = self.df[self.df.player_id.eq(player_id)]
    
    def woba_simple_static(self, player_id ,weighting = 'pitches'):
        self.select_given_player(player_id)
        wobas = np.array(self.df_this_player['woba'])
        if weighting == 'uniform':
            mean_woba = np.nanmean(wobas)
            std_woba = np.nanstd(wobas)
        else:
            weights = np.array(self.df_this_player[weighting])
            weights[np.isnan(wobas)] = 0
            if sum(weights) > 0:
                mean_woba = np.average(wobas, weights = weights)
                std_woba = np.sqrt(np.sum((wobas - mean_woba)**2 * weights) / np.sum(weights))
            else:
                mean_woba = np.nanmean(wobas)
                std_woba = np.nanstd(wobas)
            
        return mean_woba, std_woba
    
    def get_team_abb(self, player_id):
        self.select_given_player(player_id)
        abb_dict = {}
        for i in range(len(self.df_this_player)):
            this_abb = self.df_this_player['team_name_alt'].iloc[i]
            this_pa = self.df_this_player['pa'].iloc[i]
            if this_abb in abb_dict.keys():
                abb_dict[this_abb] += this_pa
            else:
                abb_dict[this_abb] = this_pa
        return abb_dict
    
    def get_isDH(self, player_id):
        if self.year >= 2022:
            return True
        abb_dict = self.get_team_abb(player_id)
        AL_pa = 0
        NL_pa = 0
        for abb in abb_dict.keys():
            if abb in league_abb_dict['NL']:
                NL_pa += abb_dict[abb]
            if abb in league_abb_dict['AL']:
                AL_pa += abb_dict[abb]
        if AL_pa > NL_pa:
            return True
        return False

In [5]:
PA = Pitcher_Arsenal(2022,'batter')

In [6]:
PA.get_isDH(543243)

True

In [7]:
#-------------------Load Useful Columns---------------------
def find_indices(list_to_check, item_to_find):
    indices = []
    for idx, value in enumerate(list_to_check):
        if value == item_to_find:
            indices.append(idx)
    return indices

def get_column_list(player_type):
    column_name_filename = '欄位分析.xlsx'
    df0 = pd.read_excel(column_name_filename, sheet_name = player_type)
    #print(df0.shape)
    Dict_mark = {df0['欄位名'][i]:df0['Mark'][i] for i in range(df0.shape[0])}
    VA = {df0['欄位名'][i]:df0['變數一'][i] for i in range(df0.shape[0])}
    VB = {df0['欄位名'][i]:df0['變數二'][i] for i in range(df0.shape[0])}
    Operator = {df0['欄位名'][i]:df0['運算子'][i] for i in range(df0.shape[0])}
    col_list_a = [list(Dict_mark.keys())[i] for i in find_indices(list(Dict_mark.values()), 'A')]
    col_list_a2 = [list(Dict_mark.keys())[i] for i in find_indices(list(Dict_mark.values()), 'A2')]
    col_list_f = [list(Dict_mark.keys())[i] for i in find_indices(list(Dict_mark.values()), 'F')]
    col_list_e = [list(Dict_mark.keys())[i] for i in find_indices(list(Dict_mark.values()), 'E')]
    col_list_g = [list(Dict_mark.keys())[i] for i in find_indices(list(Dict_mark.values()), 'G')]
    col_list_y = [list(Dict_mark.keys())[i] for i in find_indices(list(Dict_mark.values()), 'y')]
    col_dict_derived = {col:{'VA':VA[col], 'VB':VB[col], 'Operator':Operator[col]} 
                        for col in col_list_g + col_list_e}
    col_list = {'col_list_a':col_list_a, 'col_list_a2':col_list_a2, 'col_list_f':col_list_f,
               'col_dict_derived':col_dict_derived, 'col_list_y':col_list_y}
    return col_list

In [8]:
def convert_single_year(year, player_type):
    import copy
    csv_file_name = f'{csv_file_path}{year}{player_type}.csv'
    read_df = pd.read_csv(csv_file_name)
    direct_col_list = col_list_f + col_list_a + col_list_a2
    dcl = copy.deepcopy(direct_col_list)
    none_col_list = []
    if player_type == 'pitcher':
        for i in dcl:
            if not i in list(read_df.columns):
                direct_col_list.remove(i)
                none_col_list.append(i)
    
    this_df = read_df[direct_col_list]
    for i in none_col_list:
        this_df.insert(this_df.shape[1], i, None)
    for col in col_dict_derived:
        this_VA_name = col_dict_derived[col]['VA']
        this_VB_name = col_dict_derived[col]['VB']
        this_Operator = col_dict_derived[col]['Operator']
        if this_VA_name in list(read_df.columns):
            this_VA = read_df[this_VA_name]
        else:
            this_VA = this_df[this_VA_name]
        if this_VB_name in list(read_df.columns):
            this_VB = read_df[this_VB_name]
        else:
            this_VB = this_df[this_VB_name]
        if this_Operator == '+':
            this_df.insert(this_df.shape[1], col, this_VA + this_VB)
        elif this_Operator == '/':
            this_df.insert(this_df.shape[1], col, this_VA / this_VB *100)
        else:
            print('The operator is wrong.')
            
    PA = Pitcher_Arsenal(year, player_type)
    player_ids = list(this_df['player_id'])
    woba_stds = []
    isDH = []
    for player_id in player_ids:
        a, b = PA.woba_simple_static(player_id)
        woba_stds.append(b/a)
        if PA.get_isDH(player_id):
            isDH.append(1)
        else:
            isDH.append(0)
    this_df.insert(this_df.shape[1], column = "woba_std", value = woba_stds)
    this_df.insert(this_df.shape[1], column = "isDH", value = isDH)
#     if year >= 2019:
#         PA = Pitcher_Arsenal(year, player_type)
#         player_ids = list(this_df['player_id'])
#         woba_stds = []
#         for player_id in player_ids:
#             a, b = PA.woba_simple_static(player_id)
#             woba_stds.append(b/a)
#         this_df.insert(this_df.shape[1], column = "woba_std", value = woba_stds)
#     else:
#         this_df.insert(this_df.shape[1], column = "woba_std", value = None)
    for i in col_list_y:
        this_df.insert(this_df.shape[1], i, read_df[i])
    
    if player_type == 'pitcher':
        this_df = this_df.rename(columns={'p_total_pa': 'pa'})
    return this_df

In [9]:
for player_type in ['batter', 'pitcher']:
#for player_type in ['batter',]:
    col_list = get_column_list(player_type)
    col_list_y = col_list['col_list_y']
    col_list_f = col_list['col_list_f']
    col_list_a = col_list['col_list_a']
    col_list_a2 = col_list['col_list_a2']
    col_dict_derived = col_list['col_dict_derived']
    for year in range(2017, 2024):
        this_df = convert_single_year(year, player_type)
        this_df.to_csv(f'{csv_file_path}{year}_{player_type}_arranged.csv', index=False)

C:\Users\skyja\AppData\Local\Temp\ipykernel_36808\2276847684.py:42: RuntimeWarning: invalid value encountered in double_scalars
  woba_stds.append(b/a)
C:\Users\skyja\AppData\Local\Temp\ipykernel_36808\2699739637.py:25: RuntimeWarning: Mean of empty slice
  mean_woba = np.nanmean(wobas)
C:\Users\skyja\anaconda3\envs\ink\lib\site-packages\numpy\lib\nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\skyja\AppData\Local\Temp\ipykernel_36808\2276847684.py:42: RuntimeWarning: invalid value encountered in double_scalars
  woba_stds.append(b/a)
C:\Users\skyja\AppData\Local\Temp\ipykernel_36808\2699739637.py:25: RuntimeWarning: Mean of empty slice
  mean_woba = np.nanmean(wobas)
C:\Users\skyja\anaconda3\envs\ink\lib\site-packages\numpy\lib\nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\skyja\AppData\Local\T